Copyright 2018, 2019 Tobias Jachowski

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [58]:
import matplotlib
matplotlib.use('module://ipympl.backend_nbagg')

import imageio
import os
import time
import pathlib
import tifffile
import numpy as np

from matplotlib import pyplot as plt
plt.ion()
from ipywidgets import interact, IntSlider
from IPython.core.display import Image, display
from matplotlib.widgets import SpanSelector


def file_and_dir(filename=None, directory=None):
    filename = filename or ""
    fdir = os.path.dirname(filename)
    ffile = os.path.basename(filename)

    ddir = directory or "."

    if (ffile == "" or ffile == "." or ffile == ".."):
        directory = os.path.join(ddir, filename, "")
        absdir = os.path.realpath(directory)
        return None, absdir, None

    directory = os.path.join(ddir, fdir, "")
    absdir = os.path.realpath(directory)
    absfile = os.path.join(absdir, ffile)

    return ffile, absdir, absfile


def files(directory, prefix=None, suffix=None, extension=None, sort_key=None):
    """
    Get filenames of a directory in the order sorted to their filename or a
    given key function.

    Parameters
    ----------
    directory : str
        The directory the files are located in.
    prefix : str
        Get only the files beginning with `prefix`.
    suffix : str
        Get only the files ending with `suffix`.
    extension : str, optional
        The extension of the files that should be returned.
    sort_key : function
        Function to be applied to every filename found, before sorting.
    """
    prefix = prefix or ''
    suffix = suffix or ''
    extension = extension or ''
    files = [file_and_dir(filename=name, directory=directory)[2]
             for name in os.listdir(directory)
             if os.path.isfile(os.path.join(directory, name))
             and name.startswith(prefix)
             and name.endswith(''.join((suffix, extension)))]
    files.sort(key=sort_key)
    return files


def dtype_info(dtype=None, array=None):
    if array is not None:
        dtype = dtype or array.dtype
    try:
        info = np.iinfo(dtype)
    except:
        info = np.finfo(dtype)
    return dtype, info


def lookup_table(dtype, minimum=None, maximum=None, dtype_to=None):
    dtype, info = dtype_info(dtype=dtype)
    dtype_to = dtype_to or dtype
    dtype_to, info_to = dtype_info(dtype=dtype_to)

    minimum = minimum or info.min
    maximum = maximum or info.max
    minimum = max(minimum, info.min)
    maximum = min(maximum, info.max)
    
    # Create a lookup table 
    lut = np.zeros(2**info.bits, dtype=dtype_to)
    lut[:minimum] = info_to.min
    lut[minimum:maximum + 1] = np.linspace(info_to.min, info_to.max, maximum - minimum + 1, dtype=dtype_to)
    lut[maximum:] = info_to.max

    return lut


def adjust_contrast(image, min, max):
    dtype = image.dtype
    lut = lookup_table(dtype, min, max)
    return np.take(lut, image)


def convert_image(image, dtype_to, min, max):
    dtype = image.dtype
    lut = lookup_table(dtype, min, max, dtype_to)
    return np.take(lut, image)


def convert_uin16_uint8(image, minimum=None, maximum=None, width=None):
    if width is None:
        minimum = image.min() if minimum is None else minimum
        maximum = image.max() if maximum is None else maximum
    else:
        # adjust histogramm according to given width, centered around the median
        #hist, values = np.histogram(im, bins=2**16-1)
        #middle = hist.argmax()
        median = np.median(image)
        minimum = median - width / 2
        maximum = median + width / 2
    minimum = int(round(minimum))
    maximum = int(round(maximum))
    return convert_image(image, 'uint8', minimum, maximum)


class adjust_image_contrast(object):
    def __init__(self, image=None, dtype=None):
        """
        Display images of a directory with an interactive widget and a slider in a
        jupyter notebook, in the order sorted to their filename or a given key
        function.

        Parameters
        ----------
        image : np.ndarray
        dtype : np.ndarray.dtype
        """
        if dtype is not None or image is not None:
            dtype, info = dtype_info(array=image, dtype=dtype)
            self.min = info.min
            self.max = info.max
        else:
            self.min = None
            self.max = None

        self.fig, axes = plt.subplots(nrows=2, ncols=2)
        self.axes = axes.flatten()
        
        self.image = None
        if image is not None:
            self.process_image(image)

    def __call__(self, image=None):
        self.process_image(image)

    def process_image(self, image=None):
        if image is None:
            image = self.image
        else:
            self.image = image
            self.axes[0].clear()    
            self.axes[0].imshow(image, cmap=plt.cm.gray)
            self.axes[2].clear()
            self.axes[2].hist(image.ravel(), bins=(2**8 - 1))
            self.spanselector = SpanSelector(self.axes[2], self.set_min_max_hist,
                                             'horizontal', useblit=True)
            self.axspan = self.axes[2].axvspan(self.min, self.max,
                                               facecolor='y', alpha=0.2)

        self.axes[1].clear()
        self.axes[3].clear()
        self.image_contrast = adjust_contrast(image, self.min, self.max)
        im_c = self.image_contrast
        self.axes[1].imshow(im_c, cmap=plt.cm.gray)
        self.axes[3].hist(im_c.ravel(), bins=(2**8 - 1))
    
    def set_min_max_hist(self, minimum, maximum):
        """
        Set the timespan according to the SpanSelector and update the figure
        accordingly. Call all registered callback functions.

        This function is called upon any change of the SpanSelector.

        Parameters
        ----------
        min : float
        max : float
        """
        self.min = int(np.round(minimum))
        self.max = int(np.round(maximum))
        try:
            dtype, info = dtype_info(array=self.image)
            self.min = max(self.min, info.min)
            self.max = min(self.max, info.max)
        except:
            pass
        self.axspan.set_xy([[minimum, 0],  # lower left corner
                            [minimum, 1],  # upper left corner
                            [maximum, 1],  # upper right corner
                            [maximum, 0],  # lower right corner
                            [minimum, 0]])  # lower left corner
        self.process_image()


class process_images(object):
    def __init__(self, process, directory, prefix=None, suffix=None, extension='.png',
                 sort_key=None, idx_r=None, idx_c=None):
        """
        Display images of a directory with an interactive widget and a slider in a
        jupyter notebook, in the order sorted to their filename or a given key
        function.

        Parameters
        ----------
        process : function
            function, which takes an image (np.ndarray) as an argument
        directory : str
            The directory the images to be displayed are located in.
        prefix : str
            Display only the files beginning with `prefix`.
        suffix : str
            Display only the files ending with `suffix`.
        extension : str, optional
            The extension of the images that should be displayed. Default is
            '.png'.
        sort_key : function
            Function to be applied to every image filename found, before sorting.
        """
        self.images = files(directory, prefix, suffix, extension, sort_key)
        self.process = process
        self.idx_r = idx_r or slice(None)
        self.idx_c = idx_c or slice(None)

        stop = len(self.images)
        if stop < 1:
            print("No file found with prefix '%s', suffix '%s', and extension '%s'"
                  % (prefix, suffix, extension))
            return

        slider = IntSlider(min=0, max=stop-1, step=1, value=0,
                           description='Image:')
        
        def process_image(i):
            self.process_image(i)

        self.interact = interact(process_image, i=slider)
    
    def process_image(self, i):
        image = self.images[i]
        try:
            print(image)
            try:
                im = imageio.imread(image)
            except:
                im = tifffile.imread(image)
            self.process(im[self.idx_r, self.idx_c])
        except:
            print('No image found!')


def filterby(files, prefix=None, suffix=None, extension=None):
    prefix = prefix or ''
    suffix = suffix or ''
    extension = extension or ''
    for file in files:
        if file.startswith(prefix) and file.endswith(''.join((suffix, extension))):
            yield file


def get_image_shape(image):
    height, width = tifffile.imread(image).shape
    return width, height


def get_crop_image_roi(width, height, center_x=None, center_y=None, crop_x=None, crop_y=None, multiple_of=None):
    def get_start_stop(length, center, crop, multiple_of=None):
        center = 0.5 if center is None else center
        crop = 1.0 if crop is None else crop
        center = min(max(center, 0.0), 1.0)
        crop = min(max(crop, 0.0), 1.0)

        new_center = int(round((length - 1) * center))  # new_center between 0 and (length - 1)
        max_length = min(new_center, length - new_center - 1) * 2 + 1  # set max_length according to position of new_center
        new_length = min(max(int(round(length * crop)), 1), max_length)  # set new_length between 1 and (crop_length or max_length)

        # make new_length a multiple of multiple_of
        if multiple_of is not None:
            multiple_of = min(max(multiple_of, 1), max_length)
            reminder = new_length % multiple_of
            if reminder >= multiple_of / 2:
                new_length += multiple_of - reminder
            else:
                new_length -= reminder
            if new_length > max_length:
                new_length -= multiple_of
            new_length = max(new_length, 1)
 
        start = int(round((new_center - new_length * 0.5)))
        start = min(max(start, 0), length - new_length)
        stop = start + new_length
        return start, stop

    start_x, stop_x = get_start_stop(width, center_x, crop_x, multiple_of)
    start_y, stop_y = get_start_stop(height, center_y, crop_y, multiple_of)

    return (start_x, stop_x, start_y, stop_y)


def scalebar(image, resolution=1.0, width=1.0, height=None, pos_x_rel=0.98, pos_y_rel=None, value=None):
    """
    Draw a scalebar on top of an image
    
    Parameters
    ----------
    resolution : float
        The resolution of the image in unit/px
    width : float
        The width of the scalebar in units
    height : float
        The height of the scalebar in units. Defaults to 0.15*`width`.
    pos_x_rel : float
        Relative x position of image up to where the scalebar should extend.
    pos_y_rel : float
        Relative y position of image up to where the scalebar should extend.
    value : int
        The integer value of the color of the scalebar. Defaults to the
        maximal allowed value of the `image` arra. Depending on the lookup
        table this translates usually to white or black.
    """
    width_px = int(np.round(width / resolution))
    height = 0.15 * width if height is None else height
    height_px = int(np.round(height / resolution))

    image_height, image_width = image.shape
    pos_x = int(np.round(pos_x_rel * image_width))
    pos_y = image_height - (image_width - pos_x) if pos_y_rel is None else int(np.round(pos_y_rel * image_height))

    dtype, info = dtype_info(array=image)
    value = info.max if value is None else value

    image[pos_y - height_px:pos_y, pos_x - width_px:pos_x] = value


def create_video(directory, prefix=None, suffix=None, extension=None, 
                 fps=None, fps_speedup=1, decimate=1, quality=None,
                 min_grey=None, max_grey=None, width=None,
                 center_x=None, center_y=None, crop_x=None, crop_y=None,
                 resolution=1, scalebar_width=None, scalebar_height=None,
                 videoname=None, videosuffix='.mp4', videodirectory=None):
    """
    Parameters
    ----------
    fps : float or str
        Autodetect or set the frames per second of the source files. 'predominant'
        calculates the median of all creation time differences between all files
        and uses the reciprocal as fps. 'total' uses the difference of the creation
        time of the last and the first file and divides it by the total number of
        images. Defaults to 'predominant'.
    """
    # Get all files with correct prefix, suffix and extension in the directory
    for root, dirs, filenames in os.walk(directory):
        filenames = list(filterby(filenames, prefix=prefix, extension=extension))
        break

    # Create video, if the number of images is sufficient
    if len(filenames) >= 2:
        print('Creating Video of files in {} ...'.format(root))

        def creation_time(filename):
            fullname = os.path.join(root, filename)
            return os.stat(fullname).st_mtime
        filenames.sort(key=creation_time)

        def fps_explicit():
            return fps
        def fps_predominant():
            times = np.array([creation_time(f) for f in filenames])
            tdiffs = times[1:] - times[:-1]
            tdiff = np.median(tdiffs)
            fps = 1/tdiff
            return fps
        def fps_total():
            start_time = creation_time(filenames[0])
            end_time = creation_time(filenames[-1])
            duration = end_time - start_time
            fps = (len(filenames) - 1) / duration
            return fps
        fps_options = {
            None: fps_predominant,
            'predominant': fps_predominant,
            'total': fps_total
        }
        # Determine fps ('predominant' or 'total') or set directly ('fps_explicit')
        fps_source = fps_options.get(fps, fps_explicit)()
        fps_target = fps_source * fps_speedup / decimate
        print('Frames per second source: {:.2f}'.format(fps_source))
        print('Frames per second target: {:.2f}'.format(fps_target))

        if videoname is None:
            videoname = prefix
        videoname = ''.join((videoname, videosuffix))
        if videodirectory is None:
            videodirectory = os.path.join(root, '..')
        savename = os.path.join(videodirectory, videoname)
        if savename.endswith('.tif'):
            writer = imageio.get_writer(savename)
        else:
            # pixelformat='gray16le'
            writer = imageio.get_writer(savename, fps=fps_target, quality=quality)

        if center_x is None and center_y is None and crop_x is None and crop_y is None:
            roi = (None, None, None, None)
        else:
            image_width, image_height = get_image_shape(os.path.join(root, filenames[0]))
            roi = get_crop_image_roi(image_width, image_height, center_x, center_y, crop_x, crop_y,
                                     multiple_of=16)
        idx_x = slice(roi[0], roi[1])  # columns
        idx_y = slice(roi[2], roi[3])  # rows

        for filename in filenames[::decimate]:
            fullname = os.path.join(root, filename)
            #im = imageio.imread(fullname, format='tif', multifile=False)
            im = tifffile.imread(fullname)[idx_y, idx_x]
            if scalebar_width is not None:
                scalebar(im, resolution, width=scalebar_width, height=scalebar_height)
            im = convert_uin16_uint8(im, minimum=min_grey, maximum=max_grey, width=width)
            writer.append_data(im)

        writer.close()
        print('... Created video {}.'.format(videoname))

In [7]:
directory = '../images'
prefix = 'B03'
suffix = None
extension = '.tif'

# Adjust the contrast of many images by showing interactive widgets to
#   1. Create a contrast process object, to adjust the contrast
#   2. Create a process image object, to interactively show individual
#      images before and after processing
cp = adjust_image_contrast()
pi = process_images(cp, directory, prefix=prefix, suffix=suffix, extension=extension)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='Image:', max=1166), Output()), _dom_classes=('widget-int…

In [59]:
# CREATE VIDEO WITH PREFIX 'prefix' CONTAINED IN THE FOLDER 'directory'

# Adjust the contrast
width = None  # set the desired range of the grey values
minimum, maximum = cp.min, cp.max  # set the contrast min max values
print('# minimum, maximum = {}, {}  # {}, {}'.format(minimum, maximum, os.path.basename(directory), prefix))

# Autodetect fps or set explicitly
fps = None  # set the fps of the source video
fps_speedup = 1  # factor of speedup of the video
decimate = 1  # read every `decimate` image to produce the video

center_x = None
center_y = None
crop_x = None
crop_y = None

quality = None  # defaults to 5

resolution = 16  # nm/px resolution for the scalebar
scalebar_width = None  # width of the scalebar
videosuffix = '.mp4'  # select the type of video to be produced

# Create the video
create_video(directory, prefix=prefix, suffix=suffix, extension=extension,
             fps=fps, fps_speedup=fps_speedup, decimate=decimate, quality=quality,
             min_grey=minimum, max_grey=maximum, width=width,
             center_x=center_x, center_y=center_y, crop_x=crop_x, crop_y=crop_y,
             resolution=resolution, scalebar_width=scalebar_width,
             videosuffix=videosuffix)

# minimum, maximum = 32364, 33132  # images, B07
Creating Video of files in ../images ...
Frames per second: 24.80
... Created video B07.mp4.


In [ ]:
# CREATE ALL VIDEOS CONTAINED IN THE FOLDER 'directory'

# Auto adjust the contrast
width = 2000  # set the desired range of the grey values
minimum, maximum = None, None  # set the contrast min max values

# Autodetect fps or set explicitly
fps = 'total'  # set the fps of the source video
fps_speedup = 1  # factor of speedup of the target video
decimate = 1  # read every `decimate` image to produce the video

center_x = None
center_y = None
crop_x = None
crop_y = None

quality = None  # defaults to 5

resolution = 16  # nm/px resolution for the scalebar
scalebar_width = None  # width of the scalebar
videosuffix = '.mp4'  # select the type of video to be produced

# Create many videos and auto set contrast
for prefix in ['B{:02d}'.format(i+1) for i in range(7)]:
    create_video(directory, prefix=prefix, suffix=suffix, extension=extension,
                 fps=fps, fps_speedup=fps_speedup, decimate=decimate, quality=quality,
                 min_grey=minimum, max_grey=maximum, width=width,
                 center_x=center_x, center_y=center_y, crop_x=crop_x, crop_y=crop_y,
                 resolution=resolution, scalebar_width=scalebar_width,
                 videosuffix=videosuffix)

Creating Video of files in ../images ...
Frames per second: 24.63
... Created video B01.mp4.
Creating Video of files in ../images ...
Frames per second: 35.38
... Created video B02.mp4.
Creating Video of files in ../images ...
Frames per second: 26.91
